# Introduction

In this project, we will try to see if there is a relation between the intensity of care of cardiologists and
their vertical integration status.

First, we will set our environment up. We used discovery cluster for this project.


# Setting up the environment

We'll load the needed libraries:


In [2]:
setwd("/work/postresearch/Shared/Projects/Farbod")
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)

numcores=64
numcores_foreach=126

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)

setDTthreads(numcores)

library(foreach)
library(doMC)
registerDoMC(cores=numcores_foreach)

# Patient level calculations

## Reading MBSF and revenue center files

### MBSF

In [2]:
mbsf_2013=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table
mbsf_2014=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table
mbsf_2015=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table
mbsf_2016=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table
mbsf_2017=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table
mbsf_2018=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table
mbsf_2019=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table
mbsf_2020=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table


New names:
* `1` -> `1...5`
* `1` -> `1...7`
* `0` -> `0...8`
* `0` -> `0...9`
* `0` -> `0...11`
* ...

Rows: 2758309 Columns: 43

── Column specification ───────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17,...
dbl (14): 2, 1...5, 70, 1...7, 0...8, 10, 0...11, 0...12, 12...37, 12...38, ...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `1` -> `1...5`
* `1` -> `1...7`
* `0` -> `0...8`
* `0` -> `0...9`
* `0` -> `0...11`
* ...

Rows: 2836378 Columns: 43

── Column specification ───────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17,...
dbl (14): 2, 1...5, 71, 1...7, 0...8, 10, 0...11, 0...12, 12...37, 12...38, ...


ℹ Use `spec()

In [9]:
mbsf_colnames_2013_2015=c("DESY_SORT_KEY","STATE_CODE","COUNTY_CODE","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MEDICARE_STATUS_CD","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","REFERENCE_YEAR")

mbsf_colnames_2016_2020=c("DESY_SORT_KEY","REFERENCE_YEAR","SAMPLE_GROUP","STATE_CODE","COUNTY_CODE","STATE_CNTY_FIPS_CD_01","STATE_CNTY_FIPS_CD_02","STATE_CNTY_FIPS_CD_03","STATE_CNTY_FIPS_CD_04","STATE_CNTY_FIPS_CD_05","STATE_CNTY_FIPS_CD_06","STATE_CNTY_FIPS_CD_07","STATE_CNTY_FIPS_CD_08","STATE_CNTY_FIPS_CD_09","STATE_CNTY_FIPS_CD_10","STATE_CNTY_FIPS_CD_11","STATE_CNTY_FIPS_CD_12","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MDCR_STATUS_CODE_01","MDCR_STATUS_CODE_02","MDCR_STATUS_CODE_03","MDCR_STATUS_CODE_04","MDCR_STATUS_CODE_05","MDCR_STATUS_CODE_06","MDCR_STATUS_CODE_07","MDCR_STATUS_CODE_08","MDCR_STATUS_CODE_09","MDCR_STATUS_CODE_10","MDCR_STATUS_CODE_11","MDCR_STATUS_CODE_12","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","DUAL_STUS_CD_01","DUAL_STUS_CD_02","DUAL_STUS_CD_03","DUAL_STUS_CD_04","DUAL_STUS_CD_05","DUAL_STUS_CD_06","DUAL_STUS_CD_07","DUAL_STUS_CD_08","DUAL_STUS_CD_09","DUAL_STUS_CD_10","DUAL_STUS_CD_11","DUAL_STUS_CD_12")


colnames(mbsf_2013)=mbsf_colnames_2013_2015
colnames(mbsf_2014)=mbsf_colnames_2013_2015
colnames(mbsf_2015)=mbsf_colnames_2013_2015
colnames(mbsf_2016)=mbsf_colnames_2016_2020
colnames(mbsf_2017)=mbsf_colnames_2016_2020
colnames(mbsf_2018)=mbsf_colnames_2016_2020
colnames(mbsf_2019)=mbsf_colnames_2016_2020
colnames(mbsf_2020)=mbsf_colnames_2016_2020

In [25]:
mbsf_2013[DESY_SORT_KEY==100000905]

DESY_SORT_KEY,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ESRD_INDICATOR,MEDICARE_STATUS_CD,PART_A_TERMINATION_CODE,PART_B_TERMINATION_CODE,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,HI_COVERAGE,SMI_COVERAGE,HMO_COVERAGE,STATE_BUY_IN_COVERAGE,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,REFERENCE_YEAR
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
100000905,22,040,2,1,76,0,0,0,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,13


### Revenue center files

In [3]:
revenue_center_2013_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuej_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2014_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuej_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2015_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuej_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2016_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2017_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2018_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2019_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2020_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table()



New names:
* `1` -> `1...2`
* `1` -> `1...3`
* `20131105` -> `20131105...4`
* `20131105` -> `20131105...7`
* `` -> ...10
* ...

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 56454360 Columns: 32

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): 0300, 00000, 36415, ...10, ...11, ...16, A, B
dbl (23): 100000015, 1...2, 1...3, 20131105...4, 40, 20131105...7, 2, 1...13...
lgl  (1): ...15


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `1` -> `1...2`
* `1` -> `1...3`
* `20140123` -> `20140123...4`
* `20140123` -> `20140123...7`
* `` -> ...10
* ...

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 58178971 Columns: 32

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): 0510, 00634, G0463, ...10

In [25]:
colnames_revenue_center_outpatient_2013_2015=c("DESY_SORT_KEY","CLAIM_NO","CLM_LINE_NUM","CLM_THRU_DT","NCH_CLM_TYPE_CD","REV_CNTR","REV_CNTR_DT","REV_CNTR_APC_HIPPS_CD","HCPCS_CD","HCPCS_1ST_MDFR_CD","HCPCS_2ND_MDFR_CD","REV_CNTR_PMT_MTHD_IND_CD","REV_CNTR_DSCNT_IND_CD","REV_CNTR_PACKG_IND_CD","REV_CNTR_OTAF_PMT_CD","REV_CNTR_IDE_NDC_UPC_NUM","REV_CNTR_UNIT_CNT","REV_CNTR_RATE_AMT","REV_CNTR_BLOOD_DDCTBL_AMT","REV_CNTR_CASH_DDCTBL_AMT","REV_CNTR_COINSRNC_WGE_ADJSTD_C","REV_CNTR_RDCD_COINSRNC_AMT","REV_CNTR_1ST_MSP_PD_AMT","REV_CNTR_2ND_MSP_PD_AMT","REV_CNTR_PRVDR_PMT_AMT","REV_CNTR_BENE_PMT_AMT","REV_CNTR_PTNT_RSPNSBLTY_PMT","REV_CNTR_PMT_AMT_AMT","REV_CNTR_TOT_CHRG_AMT","REV_CNTR_NCVRD_CHRG_AMT","REV_CNTR_STUS_IND_CD","REV_CNTR_PRICNG_IND_CD")

colnames_revenue_center_outpatient_2016_2020=c("DESY_SORT_KEY","CLAIM_NO","CLM_LINE_NUM","CLM_THRU_DT","NCH_CLM_TYPE_CD","REV_CNTR","REV_CNTR_DT","REV_CNTR_APC_HIPPS_CD","HCPCS_CD","HCPCS_1ST_MDFR_CD","HCPCS_2ND_MDFR_CD","HCPCS_3RD_MDFR_CD","HCPCS_4TH_MDFR_CD","REV_CNTR_PMT_MTHD_IND_CD","REV_CNTR_DSCNT_IND_CD","REV_CNTR_PACKG_IND_CD","REV_CNTR_OTAF_PMT_CD","REV_CNTR_IDE_NDC_UPC_NUM","REV_CNTR_UNIT_CNT","REV_CNTR_RATE_AMT","REV_CNTR_BLOOD_DDCTBL_AMT","REV_CNTR_CASH_DDCTBL_AMT","REV_CNTR_COINSRNC_WGE_ADJSTD_C","REV_CNTR_RDCD_COINSRNC_AMT","REV_CNTR_1ST_MSP_PD_AMT","REV_CNTR_2ND_MSP_PD_AMT","REV_CNTR_PRVDR_PMT_AMT","REV_CNTR_BENE_PMT_AMT","REV_CNTR_PTNT_RSPNSBLTY_PMT","REV_CNTR_PMT_AMT_AMT","REV_CNTR_TOT_CHRG_AMT","REV_CNTR_NCVRD_CHRG_AMT","REV_CNTR_STUS_IND_CD","REV_CNTR_PRICNG_IND_CD","REV_CNTR_RNDRNG_PHYSN_UPIN","REV_CNTR_RNDRNG_PHYSN_NPI","REV_CNTR_RNDRNG_PHYSN_SPCLTY_CD","REV_CNTR_DDCTBL_COINSRNC_CD","THRPY_CAP_IND_CD1","THRPY_CAP_IND_CD2","RC_PTNT_ADD_ON_PYMT_AMT")

colnames(revenue_center_outpatient_2013)=colnames_revenue_center_outpatient_2013_2015
colnames(revenue_center_outpatient_2014)=colnames_revenue_center_outpatient_2013_2015
colnames(revenue_center_outpatient_2015)=colnames_revenue_center_outpatient_2013_2015
colnames(revenue_center_outpatient_2016)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_outpatient_2017)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_outpatient_2018)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_outpatient_2019)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_outpatient_2020)=colnames_revenue_center_outpatient_2016_2020



In [27]:
head(revenue_center_outpatient_2013)

DESY_SORT_KEY,CLAIM_NO,CLM_LINE_NUM,CLM_THRU_DT,NCH_CLM_TYPE_CD,REV_CNTR,REV_CNTR_DT,REV_CNTR_APC_HIPPS_CD,HCPCS_CD,HCPCS_1ST_MDFR_CD,HCPCS_2ND_MDFR_CD,REV_CNTR_PMT_MTHD_IND_CD,REV_CNTR_DSCNT_IND_CD,REV_CNTR_PACKG_IND_CD,REV_CNTR_OTAF_PMT_CD,REV_CNTR_IDE_NDC_UPC_NUM,REV_CNTR_UNIT_CNT,REV_CNTR_RATE_AMT,REV_CNTR_BLOOD_DDCTBL_AMT,REV_CNTR_CASH_DDCTBL_AMT,REV_CNTR_COINSRNC_WGE_ADJSTD_C,REV_CNTR_RDCD_COINSRNC_AMT,REV_CNTR_1ST_MSP_PD_AMT,REV_CNTR_2ND_MSP_PD_AMT,REV_CNTR_PRVDR_PMT_AMT,REV_CNTR_BENE_PMT_AMT,REV_CNTR_PTNT_RSPNSBLTY_PMT,REV_CNTR_PMT_AMT_AMT,REV_CNTR_TOT_CHRG_AMT,REV_CNTR_NCVRD_CHRG_AMT,REV_CNTR_STUS_IND_CD,REV_CNTR_PRICNG_IND_CD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1e+08,1,2,20131105,40,0301,20131105,00000,80048,NA,NA,2,1,0,NA,NA,1,11.63,0,0,0.00,0,0,0,7.15,0,0.00,7.15,53.9,0,A,B
1e+08,1,3,20131105,40,0301,20131105,00000,80061,NA,NA,2,1,0,NA,NA,1,18.42,0,0,0.00,0,0,0,11.32,0,0.00,11.32,63.0,0,A,B
1e+08,1,4,20131105,40,0301,20131105,00000,80076,NA,NA,2,1,0,NA,NA,1,11.23,0,0,0.00,0,0,0,6.90,0,0.00,6.90,63.8,0,A,B
1e+08,1,5,20131105,40,0301,20131105,00000,83036,NA,NA,2,1,0,NA,NA,1,13.34,0,0,0.00,0,0,0,13.07,0,0.00,13.07,40.0,0,A,B
1e+08,1,6,20131105,40,0324,20131105,00260,71020,NA,NA,1,1,0,NA,NA,1,0.00,0,0,11.09,0,0,0,43.45,0,11.09,43.45,133.0,0,X,NA
1e+08,1,7,20131105,40,0001,NA,00000,NA,NA,NA,2,NA,NA,NA,NA,0,0.00,0,0,0.00,0,0,0,0.00,0,0.00,0.00,376.4,0,A,NA


## Choosing columns and combine years

A function to choose the needed columns from our raw Medicare files and also combining all years in one file.

### Carrier, inpatient, and outpatient

In [ ]:
choose_columns = function(data_list,
                          columns = c(
                            "DESY_SORT_KEY",
                            "CLAIM_NO",
                            "LINE_NUM",
                            "CLM_THRU_DT",
                            "LINE_PLACE_OF_SRVC_CD",
                            "HCPCS_CD",
                            "LINE_ICD_DGNS_VRSN_CD",
                            "LINE_ICD_DGNS_CD",
                            "LINE_ALOWD_CHRG_AMT",
                            "PRF_PHYSN_NPI",
                            "PRVDR_SPCLTY",
                            "PRVDR_STATE_CD"
                          )) {
  require(data.table)
  data_list = lapply(data_list, function (data)
    data [, ..columns])
  result = rbindlist(data_list)
  return(result)
}

carrier_data_all_years = choose_columns(
  list(
    carrier_data_2013,
    carrier_data_2014,
    carrier_data_2015,
    carrier_data_2016,
    carrier_data_2017,
    carrier_data_2018,
    carrier_data_2019,
    carrier_data_2020
  )
)

carrier_data_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
carrier_data_all_years[,month_year := format(date, "%Y-%m")]
                     
write_fst(carrier_data_all_years, "carrier_data_all_years.fst")

outpatient_data_all_years = choose_columns(
  data_list = list(
    outpatient_data_2013,
    outpatient_data_2014,
    outpatient_data_2015,
    outpatient_data_2016,
    outpatient_data_2017,
    outpatient_data_2018,
    outpatient_data_2019,
    outpatient_data_2020
  ),
  columns = c("DESY_SORT_KEY", "CLM_THRU_DT", "CLM_TOT_CHRG_AMT")
)
outpatient_data_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
write_fst(outpatient_data_all_years, "outpatient_data_all_years.fst")

inpatient_data_all_years = choose_columns(
  data_list = list(
    inpatient_data_2013,
    inpatient_data_2014,
    inpatient_data_2015,
    inpatient_data_2016,
    inpatient_data_2017,
    inpatient_data_2018,
    inpatient_data_2019,
    inpatient_data_2020
  ),
  columns = c(
    "DESY_SORT_KEY",
    "CLM_THRU_DT",
    "CLM_TOT_CHRG_AMT",
    "CLM_DRG_CD"
  )
)
                     
inpatient_data_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
write_fst(inpatient_data_all_years, "inpatient_data_all_years.fst")
                     

### Revenue center

In [69]:
choose_columns = function(data_list,columns) {
  require(data.table)
  data_list = lapply(data_list, function (data) { data [, ..columns] } )
  result = rbindlist(data_list)
  return(result)
}

revenue_center_outpatient_all_years = choose_columns(
  list(
  revenue_center_outpatient_2013,
  revenue_center_outpatient_2014,
  revenue_center_outpatient_2015,
  revenue_center_outpatient_2016,
  revenue_center_outpatient_2017,
  revenue_center_outpatient_2018,
  revenue_center_outpatient_2019,
  revenue_center_outpatient_2020), columns=c("DESY_SORT_KEY","CLM_THRU_DT","REV_CNTR"))


revenue_center_outpatient_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
revenue_center_outpatient_all_years[,month_year := format(date, "%Y-%m")]

DESY_SORT_KEY,CLM_THRU_DT,REV_CNTR,date
<dbl>,<dbl>,<chr>,<date>
100019625,20130101,0300,2013-01-01
100019625,20130101,0301,2013-01-01
100019625,20130101,0001,2013-01-01
100020607,20130101,0260,2013-01-01
100020607,20130101,0636,2013-01-01
100020607,20130101,0001,2013-01-01
100023251,20130101,0540,2013-01-01
100023251,20130101,0540,2013-01-01
100023251,20130101,0001,2013-01-01


In [70]:
write_fst(revenue_center_outpatient_all_years, "/work/postresearch/Shared/Projects/Data_fst/revenue_center_outpatient_all_years")

In [71]:
head(revenue_center_outpatient_all_years)

DESY_SORT_KEY,CLM_THRU_DT,REV_CNTR,date,month_year
<dbl>,<dbl>,<chr>,<date>,<chr>
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0324,2013-11-05,2013-11
1e+08,20131105,0001,2013-11-05,2013-11


### MBSF
Here, we need to follow each patient across the years in the data set. So, our aproach will differ.

In [31]:
mbsf_needed_cols = c(
    "DESY_SORT_KEY"
    ,"REFERENCE_YEAR"
    ,"STATE_CODE"
    ,"COUNTY_CODE"
    ,"SEX_CODE"
    ,"RACE_CODE"
    ,"AGE"
    ,"ORIG_REASON_FOR_ENTITLEMENT"
    ,"CURR_REASON_FOR_ENTITLEMENT"
    ,"ENTITLEMENT_BUY_IN_IND01"
    ,"ENTITLEMENT_BUY_IN_IND02"
    ,"ENTITLEMENT_BUY_IN_IND03"
    ,"ENTITLEMENT_BUY_IN_IND04"
    ,"ENTITLEMENT_BUY_IN_IND05"
    ,"ENTITLEMENT_BUY_IN_IND06"
    ,"ENTITLEMENT_BUY_IN_IND07"
    ,"ENTITLEMENT_BUY_IN_IND08"
    ,"ENTITLEMENT_BUY_IN_IND09"
    ,"ENTITLEMENT_BUY_IN_IND10"
    ,"ENTITLEMENT_BUY_IN_IND11"
    ,"ENTITLEMENT_BUY_IN_IND12"
    ,"HMO_INDICATOR01"
    ,"HMO_INDICATOR02"
    ,"HMO_INDICATOR03"
    ,"HMO_INDICATOR04"
    ,"HMO_INDICATOR05"
    ,"HMO_INDICATOR06"
    ,"HMO_INDICATOR07"
    ,"HMO_INDICATOR08"
    ,"HMO_INDICATOR09"
    ,"HMO_INDICATOR10"
    ,"HMO_INDICATOR11"
    ,"HMO_INDICATOR12"
    ,"VALID_DATE_OF_DEATH_SWITCH"
    ,"DATE_OF_DEATH")

mbsf_data=list(mbsf_2013,mbsf_2014,mbsf_2015,mbsf_2016,mbsf_2017,mbsf_2018,mbsf_2019,mbsf_2020)

for (a in 1:length(mbsf_data)){
  mbsf_data[[a]]=mbsf_data[[a]][, ..mbsf_needed_cols]
}

#rename last n columns in a dataset
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}

for (a in 1:length(mbsf_data)){
  rename_last(
    mbsf_data[[a]],
    ncol(mbsf_data[[a]])-1,
    paste(colnames(mbsf_data[[a]])[2:ncol(mbsf_data[[a]])],paste("_",as.character(2012+a),sep=""),sep=""))
}

for (a in 1:length(mbsf_data)){
  mbsf_data[[a]][,DESY_SORT_KEY:=as.integer(DESY_SORT_KEY)]
}



mbsf_data = reduce(mbsf_data, function(x,y) {full_join(x,y,by="DESY_SORT_KEY")}) %>% as.data.table()
                   


In [33]:
tail(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,⋯,ENTITLEMENT_BUY_IN_IND11_2019,ENTITLEMENT_BUY_IN_IND12_2019,HMO_INDICATOR01_2019,HMO_INDICATOR02_2019,HMO_INDICATOR03_2019,HMO_INDICATOR04_2019,HMO_INDICATOR05_2019,HMO_INDICATOR06_2019,HMO_INDICATOR07_2019,HMO_INDICATOR08_2019,HMO_INDICATOR09_2019,HMO_INDICATOR10_2019,HMO_INDICATOR11_2019,HMO_INDICATOR12_2019,VALID_DATE_OF_DEATH_SWITCH_2019,DATE_OF_DEATH_2019,REFERENCE_YEAR_2020,STATE_CODE_2020,COUNTY_CODE_2020,SEX_CODE_2020,RACE_CODE_2020,AGE_2020,ORIG_REASON_FOR_ENTITLEMENT_2020,CURR_REASON_FOR_ENTITLEMENT_2020,ENTITLEMENT_BUY_IN_IND01_2020,ENTITLEMENT_BUY_IN_IND02_2020,ENTITLEMENT_BUY_IN_IND03_2020,ENTITLEMENT_BUY_IN_IND04_2020,ENTITLEMENT_BUY_IN_IND05_2020,ENTITLEMENT_BUY_IN_IND06_2020,ENTITLEMENT_BUY_IN_IND07_2020,ENTITLEMENT_BUY_IN_IND08_2020,ENTITLEMENT_BUY_IN_IND09_2020,ENTITLEMENT_BUY_IN_IND10_2020,ENTITLEMENT_BUY_IN_IND11_2020,ENTITLEMENT_BUY_IN_IND12_2020,HMO_INDICATOR01_2020,HMO_INDICATOR02_2020,HMO_INDICATOR03_2020,HMO_INDICATOR04_2020,HMO_INDICATOR05_2020,HMO_INDICATOR06_2020,HMO_INDICATOR07_2020,HMO_INDICATOR08_2020,HMO_INDICATOR09_2020,HMO_INDICATOR10_2020,HMO_INDICATOR11_2020,HMO_INDICATOR12_2020,VALID_DATE_OF_DEATH_SWITCH_2020,DATE_OF_DEATH_2020
<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
499999741,13,19,510,1,1,52,1,1,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,14,19,510,1,1,53,1,1,C,C,C,C,C,C,C,⋯,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,2020,19,510,1,1,59,1,1,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999841,13,45,610,1,1,73,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,14,45,610,1,1,74,0,0,3,3,3,3,3,3,3,⋯,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2020,45,610,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999951,13,50,120,2,1,53,1,1,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,14,50,120,2,1,54,1,1,C,C,C,C,C,C,C,⋯,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2020,50,120,2,1,60,1,1,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999961,13,38,250,1,2,75,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,14,38,250,1,2,76,0,0,3,3,3,3,3,3,3,⋯,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,2020,38,250,1,2,82,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999971,13,38,190,2,1,73,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,N

Since date of death happens only once for each patient, I will collapse the date of death and its validity to a single column.

In [176]:
mbsf_data_death_collapsed = apply(mbsf_data[,
                                            .(DATE_OF_DEATH_2013,
                                              DATE_OF_DEATH_2014,
                                              DATE_OF_DEATH_2015,
                                              DATE_OF_DEATH_2016,
                                              DATE_OF_DEATH_2017,
                                              DATE_OF_DEATH_2018,
                                              DATE_OF_DEATH_2019,
                                              DATE_OF_DEATH_2020
                                             )
                                           ], 1, function(x){ paste(x[!is.na(x)], collapse = ", ")})

mbsf_data_death_validity_collapsed = apply(mbsf_data[,
                                            .(VALID_DATE_OF_DEATH_SWITCH_2013,
                                              VALID_DATE_OF_DEATH_SWITCH_2014,
                                              VALID_DATE_OF_DEATH_SWITCH_2015,
                                              VALID_DATE_OF_DEATH_SWITCH_2016,
                                              VALID_DATE_OF_DEATH_SWITCH_2017,
                                              VALID_DATE_OF_DEATH_SWITCH_2018,
                                              VALID_DATE_OF_DEATH_SWITCH_2019,
                                              VALID_DATE_OF_DEATH_SWITCH_2020
                                             )
                                           ], 1, function(x){ paste(x[!is.na(x)], collapse = ", ")})


mbsf_data_death_collapsed[mbsf_data_death_collapsed==""]=NA
mbsf_data_death_collapsed=as.IDate(mbsf_data_death_collapsed, "%Y%m%d")

mbsf_data_death_validity_collapsed[mbsf_data_death_validity_collapsed==""]=NA

mbsf_data[, `:=`(date_of_death_collapsed = mbsf_data_death_collapsed,
              death_validity_collapsed = mbsf_data_death_validity_collapsed)]

mbsf_data[!is.na(death_validity_collapsed),death_validity_collapsed:=TRUE]
mbsf_data[is.na(death_validity_collapsed),death_validity_collapsed:=FALSE]

Warning message in `[.data.table`(mbsf_data, , `:=`(date_of_death_collapsed = mbsf_data_death_collapsed, :
“Invalid .internal.selfref detected and fixed by taking a (shallow) copy of the data.table so that := can add this new column by reference. At an earlier point, this data.table has been copied by R (or was created manually using structure() or similar). Avoid names<- and attr<- which in R currently (and oddly) may copy the whole data.table. Use set* syntax instead to avoid copying: ?set, ?setnames and ?setattr. If this message doesn't help, please report your use case to the data.table issue tracker so the root cause can be fixed or this message improved.”


In [177]:
write_fst(mbsf_data, "/work/postresearch/Shared/Projects/Data_fst/mbsf_data")

## Yearly Calculators

These are the main functions that calculate yearly expenditures for patients and their corresponding physicians.\

### Read data from choose columns



In [3]:
carrier_data_all_years = read_fst(
    "carrier_data_all_years.fst", as.data.table = T)

In [4]:
outpatient_data_all_years = read_fst(
    "outpatient_data_all_years.fst", as.data.table = T)
inpatient_data_all_years = read_fst(
    "inpatient_data_all_years.fst", as.data.table = T)

In [5]:
mbsf_data = read_fst(
  "/work/postresearch/Shared/Projects/Data_fst/mbsf_data", as.data.table = T)
revenue_center_outpatient_all_years = read_fst(
  "/work/postresearch/Shared/Projects/Data_fst/revenue_center_outpatient_all_years", as.data.table = T)

In [6]:
carrier_sample = tail(carrier_data_all_years,100000)
outpatient_sample = outpatient_data_all_years[1:100000,]
inpatient_sample = inpatient_data_all_years[1:100000,]
mbsf_sample = mbsf_data[2000000:2100000,]
revenue_center_outpatient_sample=revenue_center_outpatient_all_years[1:100000]

head(carrier_sample)
head(outpatient_sample)
head(inpatient_sample)
head(mbsf_sample)
head(revenue_center_outpatient_sample)


DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>
499903779,40366100,1,20200625,11,87086,0,R8290,8.07,1932184967,08,45,2020-06-25,2020,2020-06
499903779,40366101,1,20200716,11,G0439,0,Z0000,96.14,1588154660,50,45,2020-07-16,2020,2020-07
499903779,40366101,2,20200716,11,G0442,0,Z1389,15.03,1588154660,50,45,2020-07-16,2020,2020-07
499903779,40366101,3,20200716,11,G8427,0,Z0000,0.00,1588154660,50,45,2020-07-16,2020,2020-07
499903779,40366101,4,20200716,11,G8510,0,Z1389,0.00,1588154660,50,45,2020-07-16,2020,2020-07
499903779,40366101,5,20200716,11,1036F,0,Z1389,0.00,1588154660,50,45,2020-07-16,2020,2020-07


DESY_SORT_KEY,CLM_THRU_DT,CLM_TOT_CHRG_AMT,date
<int>,<int>,<dbl>,<date>
100000015,20130425,3647.00,2013-04-25
100000015,20130719,1417.00,2013-07-19
100000015,20131014,28.00,2013-10-14
100000019,20130523,8171.11,2013-05-23
100000019,20130529,3249.00,2013-05-29
100000019,20130603,5759.00,2013-06-03


DESY_SORT_KEY,CLM_THRU_DT,CLM_TOT_CHRG_AMT,CLM_DRG_CD,date
<int>,<int>,<dbl>,<int>,<date>
100000099,20131027,50625.95,470,2013-10-27
100000099,20131103,19818.05,920,2013-11-03
100000315,20130103,53743.47,392,2013-01-03
100000559,20131224,24659.95,871,2013-12-24
100000905,20130802,5580.58,690,2013-08-02
100001177,20130825,9964.00,687,2013-08-25


DESY_SORT_KEY,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,ENTITLEMENT_BUY_IN_IND09_2016,ENTITLEMENT_BUY_IN_IND10_2016,ENTITLEMENT_BUY_IN_IND11_2016,ENTITLEMENT_BUY_IN_IND12_2016,HMO_INDICATOR01_2016,HMO_INDICATOR02_2016,HMO_INDICATOR03_2016,HMO_INDICATOR04_2016,HMO_INDICATOR05_2016,HMO_INDICATOR06_2016,HMO_INDICATOR07_2016,HMO_INDICATOR08_2016,HMO_INDICATOR09_2016,HMO_INDICATOR10_2016,HMO_INDICATOR11_2016,HMO_INDICATOR12_2016,VALID_DATE_OF_DEATH_SWITCH_2016,DATE_OF_DEATH_2016,REFERENCE_YEAR_2017,STATE_CODE_2017,COUNTY_CODE_2017,SEX_CODE_2017,RACE_CODE_2017,AGE_2017,ORIG_REASON_FOR_ENTITLEMENT_2017,CURR_REASON_FOR_ENTITLEMENT_2017,ENTITLEMENT_BUY_IN_IND01_2017,ENTITLEMENT_BUY_IN_IND02_2017,ENTITLEMENT_BUY_IN_IND03_2017,ENTITLEMENT_BUY_IN_IND04_2017,ENTITLEMENT_BUY_IN_IND05_2017,ENTITLEMENT_BUY_IN_IND06_2017,ENTITLEMENT_BUY_IN_IND07_2017,ENTITLEMENT_BUY_IN_IND08_2017,ENTITLEMENT_BUY_IN_IND09_2017,ENTITLEMENT_BUY_IN_IND10_2017,ENTITLEMENT_BUY_IN_IND11_2017,ENTITLEMENT_BUY_IN_IND12_2017,HMO_INDICATOR01_2017,HMO_INDICATOR02_2017,HMO_INDICATOR03_2017,HMO_INDICATOR04_2017,HMO_INDICATOR05_2017,HMO_INDICATOR06_2017,HMO_INDICATOR07_2017,HMO_INDICATOR08_2017,HMO_INDICATOR09_2017,

DESY_SORT_KEY,CLM_THRU_DT,REV_CNTR,date,month_year
<dbl>,<dbl>,<chr>,<date>,<chr>
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0301,2013-11-05,2013-11
1e+08,20131105,0324,2013-11-05,2013-11
1e+08,20131105,0001,2013-11-05,2013-11


### Diagnosis codes

First, we will add diagnosis codes from ICD and Medicare:

In [7]:
  #diagnosis codes
  #from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=52850&ver=26 and https://www.aapc.com/codes/cpt-codes-range/93451-93533/10
  angio_codes=c(93451,93452,93453,93454,93455,93456,93457,93458,93459,93460,93461,93462,93463,93464
                ,93503,93505,93530,93531,93532,93533)
  #from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleid=57326&ver=13&keyword=electrocardiogram&keywordType=starts&areaId=all&docType=NCA,CAL,NCD,MEDCAC,TA,MCD,6,3,5,1,F,P&contractOption=all&sortBy=relevance&bc=1
  ecg_codes=c(93000,93005,93010,93040,93041,93042)
  #from https://scct.org/page/CardiacCTCodes include CTangio
  cardiac_ct_codes=c(75571,75572,75573,75574)
  #from https://cardiacmri.com/tech-guide/cpt-codes-relevant-to-cardiac-mri/
  cardiac_mri_codes=c(75557,75559,75561,75563,75565)
  # from https://medicarepaymentandreimbursement.com/2011/07/cardiovascular-stress-testing-cpt-93015.html and https://www.aapc.com/codes/cpt-codes-range/93000-93050/
  stress_test_codes=c(93015,93016,93017,93018)
  #from https://www.aapc.com/codes/cpt-codes-range/93303-93356/20     includes stress echo
  echocardiography_codes=c(93303,93304,93306,93307,93308,93312,93313,93314,93315,93316,93317,93318
                           ,93320,93321,93325,93350,93351,93356,93352,93355,93356)
  #from https://www.aapc.com/codes/cpt-codes-range/92920-92979/ and https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=57479#:~:text=CPT%20codes%2092928%2C%2092933%2C%2092929,are%20assigned%20to%20APC%200104.    includes balloon and stent
  angioplasty_codes=c(92920,92921,92924,92925,92928,92929,92933,92934,92937,92938,92941,92943,92944
                      ,92973,92974,92975,92978,92979,93571,93572,"C9600","C9601","C9602","C9603"
                      ,"C9604","C9605","C9606","C9607","C9608")
  #from https://www.medaxiom.com/clientuploads/webcast_handouts/Coding_for_CABG-Open_Heart_Procedures.pdf and https://www.aapc.com/codes/cpt-codes-range/33016-33999/10    did not include 33517-33530 since these are used in conjunction with 33533-33548 and not alone, did not include 33542,33545,33548 since these are also in conjunction )aneurismectomy and vsd resection
  CABG_codes=c(33510,33511,33512,33513,33514,33516,33533,33534,33535,33536)
  #from http://www.icd9data.com/2015/Volume1/390-459/430-438/default.htm and https://www.icd10data.com/ICD10CM/Codes/I00-I99/I60-I69/I63-
  stroke_icd_9_codes=c(43301,43311,43321,43331,43381,43391,43401,43411,43491)
  office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                       ,"99215")
  IHD_icd_9_codes=c(410, 411, 412,413,414)
  IHD_icd_10_codes=c("I20", "I21", "I22", "I23", "I24", "I25")
  
non_us_state_codes=c(40,54,56,57,58,59,60,61,62,63,64,65,66,97,98,99)


### Patient yearly expenditures and use of services

I will first create a function that adds conditions of interest to the data.


In [8]:
yearly_calculator_patient_conditions = function(data) {
  
  #requirements
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  require(lubridate)
  
  data %>%
    mutate(
      is_catheterization = HCPCS_CD %in% angio_codes,
      is_ecg = HCPCS_CD %in% ecg_codes,
      is_cardiac_ct = HCPCS_CD %in% cardiac_ct_codes,
      is_cardiac_mri = HCPCS_CD %in% cardiac_mri_codes,
      is_stress_test = HCPCS_CD %in% stress_test_codes,
      is_echocardiography = HCPCS_CD %in% echocardiography_codes,
      is_angioplasty = HCPCS_CD %in% angioplasty_codes,
      is_CABG = HCPCS_CD %in% CABG_codes,
      is_stable_angina = ifelse(
        LINE_ICD_DGNS_VRSN_CD == 0,
        LINE_ICD_DGNS_CD %in% c ("I208", "I209"),
        ifelse(LINE_ICD_DGNS_VRSN_CD == 9, LINE_ICD_DGNS_CD == "4139", NA)),
      is_unstable_angina = ifelse(
        LINE_ICD_DGNS_VRSN_CD == 0,
        LINE_ICD_DGNS_CD == "I200",
        ifelse(LINE_ICD_DGNS_VRSN_CD == 9, LINE_ICD_DGNS_CD == "4111", NA)),
      is_MI = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) == "I21",
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) == "410" ,NA)),
      is_cardiac_arrest = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) == "I46",
        if_else(LINE_ICD_DGNS_VRSN_CD == 9, LINE_ICD_DGNS_CD == "4275", NA)),
      is_stroke = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) == "I63",
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          LINE_ICD_DGNS_CD %in% stroke_icd_9_codes,NA)),
      is_office_visit = LINE_PLACE_OF_SRVC_CD %in% office_visit_codes,
      is_cardiology_office_vist =
        (LINE_PLACE_OF_SRVC_CD %in% office_visit_codes) &
        (PRF_PHYSN_NPI %in% c("06", "C3")),
      is_IHD = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_9_codes,NA)),
    ) %>%
    as.data.table()
}

yearly_patient_conditions=yearly_calculator_patient_conditions(carrier_sample)
head(yearly_patient_conditions)


Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:reshape’:

    stamp


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year,is_catheterization,is_ecg,is_cardiac_ct,is_cardiac_mri,is_stress_test,is_echocardiography,is_angioplasty,is_CABG,is_stable_angina,is_unstable_angina,is_MI,is_cardiac_arrest,is_stroke,is_office_visit,is_cardiology_office_vist,is_IHD
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
499903779,40366100,1,20200625,11,87086,0,R8290,8.07,1932184967,08,45,2020-06-25,2020,2020-06,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
499903779,40366101,1,20200716,11,G0439,0,Z0000,96.14,1588154660,50,45,2020-07-16,2020,2020-07,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
499903779,40366101,2,20200716,11,G0442,0,Z1389,15.03,1588154660,50,45,2020-07-16,2020,2020-07,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
499903779,40366101,3,20200716,11,G8427,0,Z0000,0.00,1588154660,50,45,2020-07-16,2020,2020-07,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
499903779,40366101,4,20200716,11,G8510,0,Z1389,0.00,1588154660,50,45,2020-07-16,2020,2020-07,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
499903779,40366101,5,20200716,11,1036F,0,Z1389,0.00,1588154660,50,45,2020-07-16,2020,2020-07,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


### Summarizing patient data
I will now summarise the data for each patient.


In [9]:
summarise_expenditures = function(data, time_frame = 365, diagnosis){
  
  data%>%
    group_by(DESY_SORT_KEY) %>%
    filter(sum(eval(parse(
      text = paste("is_", diagnosis, sep = "")
    )), na.rm = T) == T) %>%
    mutate(first_diagnosis = min(date[eval(parse(text = paste("is_", diagnosis, sep = ""))) ==
                                        T]), na.rm = T) %>%
    mutate( had_IHD = (
      date - first_diagnosis < 0 &
        first_diagnosis - date < time_frame &
        is_IHD
    ))%>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame &
             had_IHD == F,
           .preserve = T) %>%
    summarise(
      first_diagnosis = unique(first_diagnosis),
      tot_allowed_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT),
      office_visit_count = sum(na.rm = T, is_office_visit),
      cardiology_visit_count = sum(na.rm = T, is_cardiology_office_vist),
      distinct_clinicians = length(unique(PRF_PHYSN_NPI)),
      catheterization_count = sum(na.rm = T, is_catheterization),
      catheterization_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_catheterization),
      ecg_count = sum(na.rm = T, is_ecg),
      ecg_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_ecg),
      cardiac_ct_count = sum(na.rm = T, is_cardiac_ct),
      cardiac_ct_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_cardiac_ct),
      cardiac_mri_count = sum(na.rm = T, is_cardiac_mri),
      cardiac_mri_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_cardiac_mri),
      stress_test_count = sum(na.rm = T, is_stress_test),
      stress_test_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_stress_test),
      echocardiography_count = sum(na.rm = T, is_echocardiography),
      echocardiography_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_echocardiography),
      angioplasty_count = sum(na.rm = T, is_angioplasty),
      angioplasty_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_angioplasty),
      CABG_count = sum(na.rm = T, is_CABG),
      CABG_cost = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_CABG),
      stable_angina = sum(is_stable_angina, na.rm = T) > 0,
      unstable_angina = sum(is_unstable_angina, na.rm = T) > 0,
      MI = sum(is_MI, na.rm = T) > 0,
      cardiac_arrest = sum(is_cardiac_arrest, na.rm = T) > 0,
      stroke = sum(is_stroke, na.rm = T) > 0,
      icd_9_pure = ifelse(prod(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, F, T),
      icd_10_pure = ifelse(sum(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, T, F)
    ) %>%
    group_by(DESY_SORT_KEY) %>%
    mutate(
      tot_cheap_prcdr = sum(
        ecg_count,
        stress_test_count,
        echocardiography_count,
        cardiology_visit_count,na.rm = T),
      tot_expensive_prcdr = sum(
        catheterization_count,
        cardiac_ct_count,
        cardiac_mri_count,
        angioplasty_count,
        CABG_count,na.rm = T),
      year_first_diagnosed=year(first_diagnosis)                                      
    )%>%
    as.data.table()
}

summary = summarise_expenditures(yearly_patient_conditions , diagnosis = "stable_angina")
head(summary)


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed
<int>,<date>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<int>,<int>,<dbl>
499934915,2020-10-08,1111.95,0,0,5,0,0.00,2,9.71,0,0.00,0,0,0,0,0,0.00,0,0.00,0,0,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,2,0,2020
499948783,2020-11-03,1154.79,0,0,1,1,152.42,0,0.00,0,0.00,0,0,0,0,0,0.00,2,916.91,0,0,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,0,3,2020
499950613,2020-10-12,11216.55,0,0,24,1,411.78,1,8.43,0,0.00,0,0,0,0,3,158.87,6,1693.69,0,0,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,4,7,2020
499969565,2020-12-30,844.40,0,0,6,1,352.70,1,8.75,0,0.00,0,0,0,0,0,0.00,0,0.00,0,0,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,1,1,2020
499974337,2020-04-29,5008.87,0,0,16,2,319.90,5,26.07,1,307.71,0,0,0,0,0,0.00,4,1693.47,0,0,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,5,7,2020
499978789,2020-02-11,4974.48,0,0,19,0,0.00,4,33.49,0,0.00,0,0,0,0,0,0.00,0,0.00,0,0,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,4,0,2020


### Patient characteristics (MBSF)

Here, we will look for patient characteristics that might get them excluded from our analyses. Things like age, state, etc.
I will use foreach parallel computing.

In [10]:
twelve_months_after=function(data,colname){

  z=foreach (p = 1:nrow(data), .combine = rbind) %:%
      foreach (a = 0:11, .combine = cbind) %dopar% {
        paste(colname,
              format(data[p,"first_diagnosis"] %m+% months(a),"%m"),"_",
              format(data[p,"first_diagnosis"] %m+% months(a),"%Y"),sep="")
      }
  
  y=foreach(p = 1:nrow(data), .combine = rbind) %:%
      foreach (a = 0:11, .combine = cbind) %dopar% {
        as.numeric(format(data[p,"first_diagnosis"] %m+% months(a),"%Y"))
      }

  return(list(z,y))
}

monthly_characteristics_finder=function(data,colname){
  
  require(dtplyr)
  require(tidyverse)

  z=twelve_months_after(data,colname)[[1]] 
  y=twelve_months_after(data,colname)[[2]] 
  
  result = foreach (p = 1:nrow(data), .combine = rbind) %:%
      foreach (i = 1:12, .combine = cbind) %dopar% {
        if(y[p,i]<2021){
          a=as.character(z[p,i])
          b=data[p,a]
        }
        else{
          b=NA
        }
        b
      }
  return(result)
}


patient_year_of_diagnosis_characteristic_finder = function(data,var_name){
  
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  
  var=c(paste(var_name,"_",data[,"year_first_diagnosed"],sep=""))
  result=foreach (a = 1:nrow(data),.combine = rbind) %dopar%{
    data[a,var[a]]
    }
  
  return(result)
}

add_patient_characteristics = function(mbsf_data,summary_data){
  
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  data = left_join(summary_data,mbsf_data,by="DESY_SORT_KEY") %>% as.data.frame()
  
  result=data%>%
    mutate(age_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(data,"AGE"),
           state_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(data,"STATE_CODE"),
           ENTITLEMENT_BUY_IN_IND_sum=
           rowSums(monthly_characteristics_finder(data,"ENTITLEMENT_BUY_IN_IND")==3 
                   | monthly_characteristics_finder(data,"ENTITLEMENT_BUY_IN_IND")=="C"
                   ,na.rm=T),
           HMO_INDICATOR_sum=
           rowSums(monthly_characteristics_finder(data,"HMO_INDICATOR")==3 
                   | monthly_characteristics_finder(data,"HMO_INDICATOR")=="C"
                   ,na.rm=T),
           died_in_one_year_after_diagnosis=date_of_death_collapsed<first_diagnosis+365,
           died_in_two_years_after_diagnosis=date_of_death_collapsed<first_diagnosis+730
          ) %>% 
  as.data.table()
  
  result[is.na(died_in_one_year_after_diagnosis), died_in_one_year_after_diagnosis:=FALSE]
  result[is.na(died_in_two_years_after_diagnosis), died_in_two_years_after_diagnosis:=FALSE]

  result[as.IDate("20191231", "%Y%m%d")<first_diagnosis 
                , died_in_one_year_after_diagnosis:=NA]

  result[as.IDate("20181231", "%Y%m%d")<first_diagnosis 
                , died_in_two_years_after_diagnosis:=NA]
  return(result)
}


summary_with_patient_characteristics=add_patient_characteristics(mbsf_data,summary)




In [17]:
summary_with_patient_characteristics

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,HMO_INDICATOR10_2016,HMO_INDICATOR11_2016,HMO_INDICATOR12_2016,VALID_DATE_OF_DEATH_SWITCH_2016,DATE_OF_DEATH_2016,REFERENCE_YEAR_2017,STATE_CODE_2017,COUNTY_CODE_2017,SEX_CODE_2017,RACE_CODE_2017,AGE_2017,ORIG_REASON_FOR_ENTITLEMENT_2017,CURR_REASON_FOR_ENTITLEMENT_2017,ENTITLEMENT_BUY_IN_IND01_2017,ENTITLEMENT_BUY_IN_IND02_2017,ENTITLEMENT_BUY_IN_IND03_2017,ENTITLEMENT_BUY_IN_IND04_2017,ENTITLEMENT_BUY_IN_IND05_2017,ENTITLEMENT_BUY_IN_IND06_2017,ENTITLEMENT_BUY_IN_IND07_2017,ENTITLEMENT_BUY_IN_IND08_2017,ENTITLEMENT_BUY_IN_IND09_2017,ENTITLEMENT_BUY_IN_IND10_2017,ENTITLEMENT_BUY_IN_IND11_2017,ENTITLEMENT_BUY_IN_IND12_20

#### Single core MBSF
These are the same functions as above but without parallelization.

### Adding comorbidities

Here, I will add patient comorbidity scores.


In [18]:
add_comorbidity=function(data, summary_data, time_frame = 365){

  require(comorbidity)
  
  #adding comorbidities
  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY",
                                "first_diagnosis",
                                "icd_9_pure",
                                "icd_10_pure")], by = "DESY_SORT_KEY") %>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame) %>%
    as.data.table()
  
  
  comorbidity_icd_9 = comorbidity_and_phys_data %>%
    subset(icd_9_pure == T)
  
  if (nrow(comorbidity_icd_9) != 0) {
    comorbidity_icd_9 = comorbidity(
      as.data.table(comorbidity_icd_9),
      id = "DESY_SORT_KEY",
      code = "LINE_ICD_DGNS_CD",
      score = "charlson",
      icd = "icd9",
      assign0 = T
    )
  }
  else {
    comorbidity_icd_9 = data.table(
      DESY_SORT_KEY = NA,
      score = NA,
      index = NA,
      wscore = NA,
      windex = NA
    )
  }
  
  comorbidity_icd_10 = comorbidity_and_phys_data %>%
    subset(icd_10_pure == T)
  
  if (nrow(comorbidity_icd_10) != 0) {
    comorbidity_icd_10 = comorbidity(
      as.data.table(comorbidity_icd_10),
      id = "DESY_SORT_KEY",
      code = "LINE_ICD_DGNS_CD",
      score = "charlson",
      icd = "icd10",
      assign0 = T
    )
  }
  else {
    comorbidity_icd_10 = data.table(
      DESY_SORT_KEY = NA,
      score = NA,
      index = NA,
      wscore = NA,
      windex = NA
    )
  }
 
  comorbidity_all=rbind(
    comorbidity_icd_9[,c("DESY_SORT_KEY","score","index","wscore","windex")]
    ,comorbidity_icd_10[,c("DESY_SORT_KEY","score","index","wscore","windex")]
  )
  result = left_join(summary_data,
                     comorbidity_all,
                     by="DESY_SORT_KEY",) %>% as.data.table()

}

summary_with_patient_characteristics_comorbidity=add_comorbidity(data = carrier_sample, summary_data = summary_with_patient_characteristics)
summary_with_patient_characteristics_comorbidity


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,DATE_OF_DEATH_2016,REFERENCE_YEAR_2017,STATE_CODE_2017,COUNTY_CODE_2017,SEX_CODE_2017,RACE_CODE_2017,AGE_2017,ORIG_REASON_FOR_ENTITLEMENT_2017,CURR_REASON_FOR_ENTITLEMENT_2017,ENTITLEMENT_BUY_IN_IND01_2017,ENTITLEMENT_BUY_IN_IND02_2017,ENTITLEMENT_BUY_IN_IND03_2017,ENTITLEMENT_BUY_IN_IND04_2017,ENTITLEMENT_BUY_IN_IND05_2017,ENTITLEMENT_BUY_IN_IND06_2017,ENTITLEMENT_BUY_IN_IND07_2017,ENTITLEMENT_BUY_IN_IND08_2017,ENTITLEMENT_BUY_IN_IND09_2017,ENTITLEMENT_BUY_IN_IND10_2017,ENTITLEMENT_BUY_IN_IND11_2017,ENTITLEMENT_BUY_IN_IND12_2017,HMO_INDICATOR01_2017,HMO_INDICATOR02_2017,HMO_INDICATOR03_2017,HMO_INDICATOR04_2017,HMO_INDI

### Most common physicians for each patient

Now, we will find most common physicians and cardiologists for each patient.


In [13]:
#adding most common physicians
add_patient_NPI=function(data, summary_data, time_frame = 365){

  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY",
                                "first_diagnosis",
                                "icd_9_pure",
                                "icd_10_pure")], by = "DESY_SORT_KEY") %>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame) %>%
    as.data.table()
  
  patient_NPI_count_finder = function(data) {
    result = data %>%
      group_by(DESY_SORT_KEY, PRF_PHYSN_NPI) %>%
      summarise(n = n()) %>%
      arrange(.by_group = T, desc(n))
  }
  
  patient_NPI_counts = patient_NPI_count_finder(comorbidity_and_phys_data)
  
  patient_NPI_counts = left_join(patient_NPI_counts, distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]), by =
                                   "PRF_PHYSN_NPI")
  
  find_most_common = function(data) {
    data %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  find_most_common_by_specialty = function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  most_common_physician = find_most_common(patient_NPI_counts)
  #primary care = 01:general practice/ family practice:08/ internal medicine:11/ geriatrics:38
  most_common_primary_care_physician = find_most_common_by_specialty(patient_NPI_counts,
                                                                     specialty_code = c("01", "08", "11", "38"))
  most_common_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = "06")
  most_common_interventional_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = "C3")
  
  most_common_physician = data.frame(most_common_physician) %>%
    rename_with( ~ paste0("most_common_physician_", .x))
  most_common_primary_care_physician = data.frame(most_common_primary_care_physician) %>%
    rename_with( ~ paste0("most_common_primary_care_physician_", .x))
  most_common_cardiologists = data.frame(most_common_cardiologists) %>%
    rename_with( ~ paste0("most_common_cardiologist_", .x))
  most_common_interventional_cardiologists = data.frame(most_common_interventional_cardiologists) %>%
    rename_with( ~ paste0("most_common_interventional_cardiologist_", .x))
  
  summary_data = left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_cardiologists,
    by = c("DESY_SORT_KEY" = "most_common_cardiologist_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_interventional_cardiologists,
    by = c("DESY_SORT_KEY" = "most_common_interventional_cardiologist_DESY_SORT_KEY")
  )%>%
    as.data.table()

  summary_data[, year_first_diagnosis := lubridate::year(first_diagnosis)]%>%
    as.data.table()

}

summary_with_npi=add_patient_NPI(data = carrier_sample, summary_data = summary_with_patient_characteristics_comorbidity)
summary_with_npi


`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.



DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,ENTITLEMENT_BUY_IN_IND05_2017,ENTITLEMENT_BUY_IN_IND06_2017,ENTITLEMENT_BUY_IN_IND07_2017,ENTITLEMENT_BUY_IN_IND08_2017,ENTITLEMENT_BUY_IN_IND09_2017,ENTITLEMENT_BUY_IN_IND10_2017,ENTITLEMENT_BUY_IN_IND11_2017,ENTITLEMENT_BUY_IN_IND12_2017,HMO_INDICATOR01_2017,HMO_INDICATOR02_2017,HMO_INDICATOR03_2017,HMO_INDICATOR04_2017,HMO_INDICATOR05_2017,HMO_INDICATOR06_2017,HMO_INDICATOR07_2017,HMO_INDICATOR08_2017,HMO_INDICATOR09_2017,HMO_INDICATOR10_2017,HMO_INDICATOR11_2017,HMO_INDICATOR12_2017,VALID_DATE_OF_DEATH_SWITCH_2017,DATE_OF_DEATH_2017,REFERENCE_YEAR_2018,STATE_CODE_2018,COUNTY_CODE_2018,SEX_CODE_2018,RACE_CODE_2018,AGE

## Physician integration status

Here, I will find which physicians are integrated.


In [14]:
#calculate and add physician integration data
#this only uses visits to see if a physician is integrated or not (codde list)

physician_integration_finder = function(data,
                                        integrated_place_of_service_codes = c("19", "22"),
                                        all_place_of_service_codes = c("11", "19", "22"),
                                        integration_threshold = 0.5,
                                        code_list = c(
                                          "99201",
                                          "99202",
                                          "99203",
                                          "99204",
                                          "99205",
                                          "99211",
                                          "99212",
                                          "99213",
                                          "99214",
                                          "99215"
                                        )) {
  data = subset(data, HCPCS_CD %in% code_list)
  result = data %>%
    mutate(
      is_facility = LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes,
      is_all = LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes,
    ) %>%
    group_by(PRF_PHYSN_NPI, month_year) %>%
    summarise(
      in_facility_count = sum(is_facility, na.rm = T),
      in_all_count = sum(is_all, na.rm = T),
      tot = n(),
    ) %>%
    mutate(
      in_facility_prp = in_facility_count / in_all_count,
      in_facility_prp_from_tot = in_facility_count / tot,
      is_integrated = in_facility_prp >= integration_threshold,
      is_integrated_from_tot = in_facility_prp_from_tot >= integration_threshold,
    )%>%
   as.data.table()
  result=result[,.(PRF_PHYSN_NPI,month_year,is_integrated)]
  result=reshape(result, idvar = "PRF_PHYSN_NPI", timevar = "month_year", direction = "wide")
  setcolorder(result,order(colnames(result)))
  setcolorder(result,"PRF_PHYSN_NPI")
  
}

physician_integration_stats = physician_integration_finder(carrier_sample, integration_threshold = 0.5)
head(physician_integration_stats)


`summarise()` has grouped output by 'PRF_PHYSN_NPI'. You can override using the `.groups` argument.



PRF_PHYSN_NPI,is_integrated.2020-01,is_integrated.2020-02,is_integrated.2020-03,is_integrated.2020-04,is_integrated.2020-05,is_integrated.2020-06,is_integrated.2020-07,is_integrated.2020-08,is_integrated.2020-09,is_integrated.2020-10,is_integrated.2020-11,is_integrated.2020-12
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1003013723,NA,FALSE,NA,NA,NA,NA,NA,FALSE,NA,NA,NA,NA
1003023581,NA,NA,NA,NA,FALSE,FALSE,NA,NA,NA,FALSE,NA,NA
1003032202,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,FALSE,FALSE
1003042490,NA,NA,NA,NA,NA,NA,NA,FALSE,FALSE,NA,NA,NA
1003071010,NA,NA,NA,NA,NA,NA,FALSE,NA,FALSE,FALSE,NA,NA
1003084237,NA,FALSE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


This function will add the integration status of most common physicians to each patient's summary data.



In [15]:
#rename columns
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status=function(data, physician_integration_stats){
  
  data = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  
  rename_last(
    data,
    ncol(physician_integration_stats)-1,
    paste("most_common_physician_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
    
    
  data = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()

  rename_last(
    data,
    ncol(physician_integration_stats)-1,
    paste("most_common_primary_care_physician_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  
  
  data = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_cardiologist_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  
  rename_last(
    data,
    ncol(physician_integration_stats)-1,
    paste("most_common_cardiologist_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  
  
  data = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_interventional_cardiologist_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  
  rename_last(
    data,
    ncol(physician_integration_stats)-1,
    paste("most_common_interventional_cardiologist_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  
  return(as.data.table(data))
}

add_integration_status(data = summary_with_npi, physician_integration_stats = physician_integration_stats)


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,HMO_INDICATOR07_2018,HMO_INDICATOR08_2018,HMO_INDICATOR09_2018,HMO_INDICATOR10_2018,HMO_INDICATOR11_2018,HMO_INDICATOR12_2018,VALID_DATE_OF_DEATH_SWITCH_2018,DATE_OF_DEATH_2018,REFERENCE_YEAR_2019,STATE_CODE_2019,COUNTY_CODE_2019,SEX_CODE_2019,RACE_CODE_2019,AGE_2019,ORIG_REASON_FOR_ENTITLEMENT_2019,CURR_REASON_FOR_ENTITLEMENT_2019,ENTITLEMENT_BUY_IN_IND01_2019,ENTITLEMENT_BUY_IN_IND02_2019,ENTITLEMENT_BUY_IN_IND03_2019,ENTITLEMENT_BUY_IN_IND04_2019,ENTITLEMENT_BUY_IN_IND05_2019,ENTITLEMENT_BUY_IN_IND06_2019,ENTITLEMENT_BUY_IN_IND07_2019,ENTITLEMENT_BUY_IN_IND08_2019,ENTITLEMENT_BUY_IN_IND09_2019,ENTITLEMENT_BUY_IN_IND10

## Combining all results

Here, I will combine the results of all functions above.

In [16]:
yearly_calculator = function(data,mbsf_data,diagnosis){

  require(tidyverse)
  require(lubridate)
  require(dtplyr)

  physician_integration_stats = 
  physician_integration_finder(
    data,
    integration_threshold = 0.5)


  result = data %>%
    yearly_calculator_patient_conditions()%>%
    summarise_expenditures(diagnosis = diagnosis) %>%
    add_patient_characteristics(mbsf_data = mbsf_data, summary_data = .)%>%
    add_comorbidity(data = data, summary_data = .) %>%
    add_patient_NPI(data = data, summary_data = .) %>%
    add_integration_status(
      data = .,
      physician_integration_stats = physician_integration_stats)
  
  return(list(result, physician_integration_stats))

}




### Stable angina

In [17]:
yearly_calcualtions_carrier_stable_angina=yearly_calculator(carrier_data_all_years,mbsf_data,diagnosis="stable_angina")

`summarise()` has grouped output by 'PRF_PHYSN_NPI'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.



In [27]:
lapply(yearly_calcualtions_carrier_stable_angina,tail)
nrow(yearly_calcualtions_carrier_stable_angina[[1]])
nrow(yearly_calcualtions_carrier_stable_angina[[2]])
tail(yearly_calcualtions_carrier_stable_angina[[1]]$ENTITLEMENT_BUY_IN_IND_sum)

[[1]]
   DESY_SORT_KEY first_diagnosis tot_allowed_carrier office_visit_count
1:     499986215      2015-04-07             1775.54                  0
2:     499989229      2013-04-02             5603.43                  0
3:     499989635      2013-03-05             3673.23                  0
4:     499991079      2015-05-19             4827.70                  0
5:     499994685      2014-01-14             2760.51                  0
6:     499995525      2016-05-11             2171.81                  0
   cardiology_visit_count distinct_clinicians catheterization_count
1:                      0                  10                     1
2:                      0                  19                     0
3:                      0                  12                     0
4:                      0                  22                     0
5:                      0                   8                     0
6:                      0                   7                     1
   catheterization_cost ecg_count ecg_cost cardiac_ct_count cardiac_ct_cost
1:               390.80         2    19.23                0               0
2:                 0.00         3    44.92                0               0
3:                 0.00         1     8.82                0               0
4:                 0.00         3    39.62                0               0
5:                 0.00         1    15.72                0               0
6:               154.67         2    24.69                0               0
   cardiac_mri_count cardiac_mri_cost stress_test_count stress_test_cost
1:                 0                0                 0                0
2:                 0                0                 0                0
3:                 0                0                 0                0
4:                 0                0                 0                0
5:                 0                0                 0                0
6:                 0                0                 0                0
   echocardiography_count echocardiography_cost angioplasty_count
1:                      1                 70.88                 0
2:                      0                  0.00                 0
3:                      0                  0.00                 0
4:                      2                123.52                 0
5:                      6                209.08                 0
6:                      0                  0.00                 1
   angioplasty_cost CABG_count CABG_cost stable_angina unstable_angina    MI
1:             0.00          0         0          TRUE           FALSE FALSE
2:             0.00          0         0          TRUE           FALSE FALSE
3:             0.00          0         0          TRUE           FALSE FALSE
4:             0.00          0         0          TRUE           FALSE FALSE
5:             0.00          0         0          TRUE           FALSE FALSE
6:           603.35          0         0          TRUE            TRUE FALSE
   cardiac_arrest stroke icd_9_pure icd_10_pure tot_cheap_prcdr
1:          FALSE  FALSE      FALSE       FALSE               3
2:          FALSE  FALSE       TRUE       FALSE               3
3:          FALSE   TRUE       TRUE       FALSE               1
4:          FALSE  FALSE      FALSE       FALSE               5
5:          FALSE  FALSE       TRUE       FALSE               7
6:          FALSE  FALSE      FALSE        TRUE               2
   tot_expensive_prcdr year_first_diagnosed REFERENCE_YEAR_2013 STATE_CODE_2013
1:                   1                 2015                  13              33
2:                   0                 2013                  13              10
3:                   0                 2013                  13              31
4:                   0                 2015                  13              01
5:                   0                 2014                  13              37
6:                   2                 201

[1] 38426

[1] 910654

[1] 12 12  1 12 12 12

In [22]:
write_fst(yearly_calcualtions_carrier_stable_angina[[1]],
          "results_feb/yearly_calcualtions_carrier_stable_angina.fst")

write_fst(yearly_calcualtions_carrier_stable_angina[[2]],
          "results_feb/physician_integration_stats.fst")

### Unstable angina

In [24]:
yearly_calcualtions_carrier_unstable_angina=yearly_calculator(carrier_data_all_years,mbsf_data,diagnosis="unstable_angina")

In [25]:
lapply(yearly_calcualtions_carrier_unstable_angina,tail)
nrow(yearly_calcualtions_carrier_unstable_angina[[1]])
nrow(yearly_calcualtions_carrier_unstable_angina[[2]])

[[1]]
   DESY_SORT_KEY first_diagnosis tot_allowed_carrier office_visit_count
1:     499982955      2015-12-09             7426.44                  0
2:     499983819      2017-10-06            21831.56                  0
3:     499985179      2015-03-26            13127.34                  0
4:     499986915      2013-12-03             1755.41                  0
5:     499990779      2015-04-01             4448.03                  0
6:     499991509      2018-01-25             2244.17                  0
   cardiology_visit_count distinct_clinicians catheterization_count
1:                      0                  33                     1
2:                      0                  53                     1
3:                      0                  41                     1
4:                      0                   8                     0
5:                      0                  17                     0
6:                      0                  16                     0
   catheterization_cost ecg_count ecg_cost cardiac_ct_count cardiac_ct_cost
1:               295.30         3    40.72                0               0
2:               130.41         7    74.40                0               0
3:               335.21         4    60.72                0               0
4:                 0.00         5    45.60                0               0
5:                 0.00         0     0.00                0               0
6:                 0.00         3    36.33                0               0
   cardiac_mri_count cardiac_mri_cost stress_test_count stress_test_cost
1:                 0                0                 0              0.0
2:                 0                0                 0              0.0
3:                 0                0                 0              0.0
4:                 0                0                 1             66.8
5:                 0                0                 0              0.0
6:                 0                0                 0              0.0
   echocardiography_count echocardiography_cost angioplasty_count
1:                      1                 61.91                 0
2:                      2                 73.02                 0
3:                      0                  0.00                 0
4:                      1                195.34                 0
5:                      1                 64.92                 0
6:                      3                  0.00                 0
   angioplasty_cost CABG_count CABG_cost stable_angina unstable_angina    MI
1:                0          0      0.00          TRUE            TRUE FALSE
2:                0          3   2154.05         FALSE            TRUE FALSE
3:                0          0      0.00         FALSE            TRUE FALSE
4:                0          0      0.00          TRUE            TRUE  TRUE
5:                0          0      0.00         FALSE            TRUE FALSE
6:                0          0      0.00         FALSE            TRUE FALSE
   cardiac_arrest stroke icd_9_pure icd_10_pure tot_cheap_prcdr
1:          FALSE  FALSE      FALSE        TRUE               4
2:           TRUE   TRUE      FALSE        TRUE               9
3:          FALSE  FALSE      FALSE       FALSE               4
4:          FALSE  FALSE       TRUE       FALSE               7
5:          FALSE   TRUE      FALSE       FALSE               1
6:          FALSE  FALSE      FALSE        TRUE               6
   tot_expensive_prcdr year_first_diagnosed REFERENCE_YEAR_2013 STATE_CODE_2013
1:                   1                 2015                  13              16
2:                   4                 2017                  13              11
3:                   1                 2015                  13              23
4:                   0                 2013                  13              51
5:                   0                 2015                  13              45
6:                   0                 201

[1] 23479

[1] 910654

In [27]:
write_fst(yearly_calcualtions_carrier_unstable_angina[[1]],
          "results_feb/yearly_calcualtions_carrier_unstable_angina.fst")

### Reading the results for patient yearly carrier calculations and physician integratiin statuses
This is if I have already done the steps above.

In [29]:
yearly_calcualtions_carrier_stable_angina = 
read_fst(path = "results_feb/yearly_calcualtions_carrier_stable_angina.fst"
        ,as.data.table = T)

yearly_calcualtions_carrier_unstable_angina = 
read_fst(path = "results_feb/yearly_calcualtions_carrier_unstable_angina.fst"
        ,as.data.table = T)

physician_integration_stats = 
read_fst("results_feb/physician_integration_stats.fst"
        ,as.data.table = T)

head(yearly_calcualtions_carrier_stable_angina)
head(yearly_calcualtions_carrier_unstable_angina)
head(physician_integration_stats)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,most_common_cardiologist_is_integrated.2016-07,most_common_cardiologist_is_integrated.2016-08,most_common_cardiologist_is_integrated.2016-09,most_common_cardiologist_is_integrated.2016-10,most_common_cardiologist_is_integrated.2016-11,most_common_cardiologist_is_integrated.2016-12,most_common_cardiologist_is_integrated.2017-01,most_common_cardiologist_is_integrated.2017-02,most_common_cardiologist_is_integrated.2017-03,most_common_cardiologist_is_integrated.2017-04,most_common_cardiologist_is_integrated.2017-05,most_common_cardiologist_is_integrated.2017-06,most_common_cardiologist_is_integrated.2017-07,most_common_cardi

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,most_common_cardiologist_is_integrated.2016-07,most_common_cardiologist_is_integrated.2016-08,most_common_cardiologist_is_integrated.2016-09,most_common_cardiologist_is_integrated.2016-10,most_common_cardiologist_is_integrated.2016-11,most_common_cardiologist_is_integrated.2016-12,most_common_cardiologist_is_integrated.2017-01,most_common_cardiologist_is_integrated.2017-02,most_common_cardiologist_is_integrated.2017-03,most_common_cardiologist_is_integrated.2017-04,most_common_cardiologist_is_integrated.2017-05,most_common_cardiologist_is_integrated.2017-06,most_common_cardiologist_is_integrated.2017-07,most_common_cardi

PRF_PHYSN_NPI,is_integrated.2013-01,is_integrated.2013-02,is_integrated.2013-03,is_integrated.2013-04,is_integrated.2013-05,is_integrated.2013-06,is_integrated.2013-07,is_integrated.2013-08,is_integrated.2013-09,is_integrated.2013-10,is_integrated.2013-11,is_integrated.2013-12,is_integrated.2014-01,is_integrated.2014-02,is_integrated.2014-03,is_integrated.2014-04,is_integrated.2014-05,is_integrated.2014-06,is_integrated.2014-07,is_integrated.2014-08,is_integrated.2014-09,is_integrated.2014-10,is_integrated.2014-11,is_integrated.2014-12,is_integrated.2015-01,is_integrated.2015-02,is_integrated.2015-03,is_integrated.2015-04,is_integrated.2015-05,is_integrated.2015-06,is_integrated.2015-07,is_integrated.2015-08,is_integrated.2015-09,is_integrated.2015-10,is_integrated.2015-11,is_integrated.2015-12,is_integrated.2016-01,is_integrated.2016-02,is_integrated.2016-03,is_integrated.2016-04,is_integrated.2016-05,is_integrated.2016-06,is_integrated.2016-07,is_integrated.2016-08,is_integrated.2016-09,is_integrated.2016-10,is_integrated.2016-11,is_integrated.2016-12,is_integrated.2017-01,is_integrated.2017-02,is_integrated.2017-03,is_integrated.2017-04,is_integrated.2017-05,is_integrated.2017-06,is_integrated.2017-07,is_integrated.2017-08,is_integrated.2017-09,is_integrated.2017-10,is_integrated.2017-11,is_integrated.2017-12,is_integrated.2018-01,is_integrated.2018-02,is_integrated.2018-03,is_integrated.2018-04,is_integrated.2018-05,is_integrated.2018-06,is_integrated.2018-07,is_integrated.2018-08,is_integrated.2018-09,is_integrated.2018-10,is_integrated.2018-11,is_integrated.2018-12,is_integrated.2019-01,is_integrated.2019-02,is_integrated.2019-03,is_integrated.2019-04,is_integrated.2019-05,is_integrated.2019-06,is_integrated.2019-07,is_integrated.2019-08,is_integrated.2019-09,is_integrated.2019-10,is_integrated.2019-11,is_integrated.2019-12,is_integrated.2020-01,is_integrated.2020-02,is_integrated.2020-03,is_integrated.2020-04,is_integrated.2020-05,is_integrated.2020-06,is_integrated.2020-07,is_integrated.2020-08,is_integrated.2020-09,is_integrated.2020-10,is_integrated.2020-11,is_integrated.2020-12
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NA,FALSE,NA,NA,NA,FALSE,FALSE,FALSE,NA,NA,NA,NA,NA,FALSE,NA,FALSE,NA,NA,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NA,NA,NA,FALSE,NA,FALSE,FALSE,NA,FALSE,NA,NA,NA,NA,NA,FALSE,NA,NA,NA,NA,NA,NA,FALSE,NA,NA,NA,NA,FALSE,NA,FALSE,NA,NA,FALSE,NA,NA,NA,FALSE,NA,NA,NA,NA,NA,NA,NA,FALSE,FALSE,NA,FALSE,NA,NA,NA,NA,NA,FALSE,NA,NA,NA,NA,NA
1003000142,NA,NA,NA,NA,NA,NA,FALSE,NA,TRUE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,FALSE,NA,NA,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,FALSE,NA,TRUE,FALSE,NA,NA,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NA,TRUE,TRUE,NA,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NA,NA,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
1003000159,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1003000423,NA,NA,FALSE,NA,NA,NA,NA,NA,NA,NA,FALSE,NA,NA,NA,NA,NA,NA,

## Adding inpatient and outpatient expenditure data

Adding the data to the yearly calculations with an inner join so I will only have data for stable/unstable angina patients. I will also filter based on date in this part.


In [30]:
outpatient_data_all_years_stable_angina=inner_join(outpatient_data_all_years, yearly_calcualtions_carrier_stable_angina,by="DESY_SORT_KEY")%>%as.data.table()
intpatient_data_all_years_stable_angina=inner_join(inpatient_data_all_years, yearly_calcualtions_carrier_stable_angina,by="DESY_SORT_KEY")%>%as.data.table()

outpatient_data_all_years_unstable_angina=inner_join(outpatient_data_all_years, yearly_calcualtions_carrier_unstable_angina,by="DESY_SORT_KEY")%>%as.data.table()
intpatient_data_all_years_unstable_angina=inner_join(inpatient_data_all_years, yearly_calcualtions_carrier_unstable_angina,by="DESY_SORT_KEY")%>%as.data.table()

This will take the outpatient and inpatient data and retun the total outpatient and inpatient expenditure. It also asks for the time frame which will be compared to the date first diagnosed. I will keep using yearly here.

In [31]:
#outpatient
yearly_tot_outpatient=function(data,time_frame=365){
  data%>%
    filter(date>=first_diagnosis & date-first_diagnosis<time_frame)%>%
    group_by(DESY_SORT_KEY)%>%
    summarise(tot_allowed_outpatient=sum(CLM_TOT_CHRG_AMT))%>%
    as.data.table()
}


#inpatient
yearly_calculator_inpatient=function (data, time_frame=365){
  data %>%
    filter(date>=first_diagnosis & date-first_diagnosis<time_frame)%>%
    group_by(DESY_SORT_KEY)%>%
    summarise(tot_allowed_inpatient=sum(CLM_TOT_CHRG_AMT),
              number_of_hospitalizations=length(unique(date)[is.na(CLM_DRG_CD)==F]),
    ) %>%
    mutate(was_hospitalized=number_of_hospitalizations>0)%>%
    as.data.table()
}



outpatient_tot_yearly_stable_angina=yearly_tot_outpatient(outpatient_data_all_years_stable_angina)
inpatient_tot_yearly_stable_angina=yearly_calculator_inpatient(intpatient_data_all_years_stable_angina)

outpatient_tot_yearly_unstable_angina=yearly_tot_outpatient(outpatient_data_all_years_unstable_angina)
inpatient_tot_yearly_unstable_angina=yearly_calculator_inpatient(intpatient_data_all_years_unstable_angina)



### Adding the outpatient and inpatient results to the carrier results
I will also find the total expenditure in one year

In [32]:
#stable angina

yearly_calculations_stable_angina=left_join(yearly_calcualtions_carrier_stable_angina, outpatient_tot_yearly_stable_angina, by="DESY_SORT_KEY")%>%as.data.table()
yearly_calculations_stable_angina=left_join(yearly_calculations_stable_angina, inpatient_tot_yearly_stable_angina, by="DESY_SORT_KEY")%>%as.data.table()
#finding the total expenditure in one year
yearly_calculations_stable_angina[,total_exp:=sum(tot_allowed_carrier,tot_allowed_outpatient,tot_allowed_inpatient,na.rm = T),by=DESY_SORT_KEY]
yearly_calculations_stable_angina[is.na(number_of_hospitalizations)==T,`:=`(number_of_hospitalizations=0,was_hospitalized=0)]
yearly_calculations_stable_angina[is.na(tot_allowed_outpatient)==T,`:=`(tot_allowed_outpatient=0)]
yearly_calculations_stable_angina[is.na(tot_allowed_inpatient)==T,`:=`(tot_allowed_inpatient=0)]


#unstable angina
yearly_calculations_unstable_angina=left_join(yearly_calcualtions_carrier_unstable_angina, outpatient_tot_yearly_unstable_angina, by="DESY_SORT_KEY")%>%as.data.table()
yearly_calculations_unstable_angina=left_join(yearly_calculations_unstable_angina, inpatient_tot_yearly_unstable_angina, by="DESY_SORT_KEY")%>%as.data.table()
#finding the total expenditure in one year
yearly_calculations_unstable_angina[,total_exp:=sum(tot_allowed_carrier,tot_allowed_outpatient,tot_allowed_inpatient,na.rm = T),by=DESY_SORT_KEY]
yearly_calculations_unstable_angina[is.na(number_of_hospitalizations)==T,`:=`(number_of_hospitalizations=0,was_hospitalized=0)]
yearly_calculations_unstable_angina[is.na(tot_allowed_outpatient)==T,`:=`(tot_allowed_outpatient=0)]
yearly_calculations_unstable_angina[is.na(tot_allowed_inpatient)==T,`:=`(tot_allowed_inpatient=0)]

In [33]:
head(yearly_calculations_stable_angina)
head(yearly_calculations_unstable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,most_common_cardiologist_is_integrated.2016-12,most_common_cardiologist_is_integrated.2017-01,most_common_cardiologist_is_integrated.2017-02,most_common_cardiologist_is_integrated.2017-03,most_common_cardiologist_is_integrated.2017-04,most_common_cardiologist_is_integrated.2017-05,most_common_cardiologist_is_integrated.2017-06,most_common_cardiologist_is_integrated.2017-07,most_common_cardiologist_is_integrated.2017-08,most_common_cardiologist_is_integrated.2017-09,most_common_cardiologist_is_integrated.2017-10,most_common_cardiologist_is_integrated.2017-11,most_common_cardiologist_is_integrated.2017-12,most_common_cardi

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,cardiology_visit_count,distinct_clinicians,catheterization_count,catheterization_cost,ecg_count,ecg_cost,cardiac_ct_count,cardiac_ct_cost,cardiac_mri_count,cardiac_mri_cost,stress_test_count,stress_test_cost,echocardiography_count,echocardiography_cost,angioplasty_count,angioplasty_cost,CABG_count,CABG_cost,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,HMO_INDICATOR01_2015,HMO_INDICATOR02_2015,HMO_INDICATOR03_2015,HMO_INDICATOR04_2015,HMO_INDICATOR05_2015,HMO_INDICATOR06_2015,HMO_INDICATOR07_2015,HMO_INDICATOR08_2015,HMO_INDICATOR09_2015,HMO_INDICATOR10_2015,HMO_INDICATOR11_2015,HMO_INDICATOR12_2015,VALID_DATE_OF_DEATH_SWITCH_2015,DATE_OF_DEATH_2015,REFERENCE_YEAR_2016,STATE_CODE_2016,COUNTY_CODE_2016,SEX_CODE_2016,RACE_CODE_2016,AGE_2016,ORIG_REASON_FOR_ENTITLEMENT_2016,CURR_REASON_FOR_ENTITLEMENT_2016,ENTITLEMENT_BUY_IN_IND01_2016,ENTITLEMENT_BUY_IN_IND02_2016,ENTITLEMENT_BUY_IN_IND03_2016,ENTITLEMENT_BUY_IN_IND04_2016,ENTITLEMENT_BUY_IN_IND05_2016,ENTITLEMENT_BUY_IN_IND06_2016,ENTITLEMENT_BUY_IN_IND07_2016,ENTITLEMENT_BUY_IN_IND08_2016,⋯,most_common_cardiologist_is_integrated.2016-12,most_common_cardiologist_is_integrated.2017-01,most_common_cardiologist_is_integrated.2017-02,most_common_cardiologist_is_integrated.2017-03,most_common_cardiologist_is_integrated.2017-04,most_common_cardiologist_is_integrated.2017-05,most_common_cardiologist_is_integrated.2017-06,most_common_cardiologist_is_integrated.2017-07,most_common_cardiologist_is_integrated.2017-08,most_common_cardiologist_is_integrated.2017-09,most_common_cardiologist_is_integrated.2017-10,most_common_cardiologist_is_integrated.2017-11,most_common_cardiologist_is_integrated.2017-12,most_common_cardi

Saving the results.

In [34]:
write_fst(yearly_calculations_stable_angina,
          "results_feb/yearly_calculations_stable_angina.fst") 
write_fst(yearly_calculations_unstable_angina,
          "results_feb/yearly_calculations_unstable_angina.fst")

#I am also saving the csv to be able to share the results with colleagues later.
write.csv(yearly_calculations_stable_angina,
          "results_feb/yearly_calculations_stable_angina.csv") 
write.csv(yearly_calculations_unstable_angina,
          "results_feb/yearly_calculations_unstable_angina.csv")



# Comparisons and analyses

## Comparisons between patients treated by integrated vs independent docotrs

I will compare the patients who were treated by the integrated vs non-integrated physicians, cardiologists and interventionists during the study period. Year 2013 did not divide cardiologists and inteventionists. So, We will not include this year in our comparisons of cardiologists and inteventionists.



In [3]:
yearly_calculations_stable_angina =
read_fst("results_feb/yearly_calculations_stable_angina.fst"
         ,as.data.table = T) 
yearly_calculations_unstable_angina =
read_fst("results_feb/yearly_calculations_unstable_angina.fst"
         ,as.data.table = T)
physician_integration_stats = 
read_fst("results_feb/physician_integration_stats.fst"
        ,as.data.table = T)

In [4]:
sum(is.na(yearly_calculations_unstable_angina[,most_common_cardiologist_PRF_PHYSN_NPI]))/nrow(yearly_calculations_unstable_angina)*100
sum(is.na(yearly_calculations_stable_angina[,most_common_cardiologist_PRF_PHYSN_NPI]))/nrow(yearly_calculations_stable_angina)*100

[1] 4.335789

[1] 9.087597

In [39]:
sum(is.na(yearly_calculations_unstable_angina[,most_common_interventional_cardiologist_PRF_PHYSN_NPI]))/nrow(yearly_calculations_unstable_angina)*100
sum(is.na(yearly_calculations_stable_angina[,most_common_interventional_cardiologist_PRF_PHYSN_NPI]))/nrow(yearly_calculations_stable_angina)*100

[1] 56.78692

[1] 65.86426

In [15]:
nrow(distinct(yearly_calculations_unstable_angina["most_common_cardiologist_PRF_PHYSN_NPI"]))/(nrow(distinct(yearly_calculations_unstable_angina["most_common_interventional_cardiologist_PRF_PHYSN_NPI"]))+nrow(distinct(yearly_calculations_unstable_angina["most_common_cardiologist_PRF_PHYSN_NPI"])))

[1] 0.7783468

In [21]:
table(is.na(yearly_calculations_unstable_angina["most_common_cardiologist_PRF_PHYSN_NPI"]))
table(is.na(yearly_calculations_unstable_angina["most_common_interventional_cardiologist_PRF_PHYSN_NPI"]))


table(is.na(yearly_calculations_unstable_angina["most_common_cardiologist_PRF_PHYSN_NPI"])+is.na(yearly_calculations_unstable_angina["most_common_interventional_cardiologist_PRF_PHYSN_NPI"]))




FALSE  TRUE 
22453  1018 


FALSE  TRUE 
10146 13325 


    0     1     2 
10096 12407   968 

In [ ]:
intensity_comparator=function(data,grouping_var){
  require(tidyverse)
  result=data%>%
    group_by(eval(parse(text=grouping_var)))%>%
    summarise(
      n=n(),
      total_exp=sum(na.rm=T,total_exp)/n(),
      tot_allowed_inpatient=sum(na.rm=T,tot_allowed_inpatient)/n(),
      tot_allowed_outpatient=sum(na.rm=T,tot_allowed_outpatient)/n(),
      tot_allowed_carrier=sum(na.rm=T,tot_allowed_carrier)/n(),
      stable_angina=sum(na.rm=T,stable_angina)/n(),
      unstable_angina=sum(na.rm=T,unstable_angina)/n(),
      MI=sum(na.rm=T,MI)/n(),
      cardiac_arrest=sum(na.rm=T,cardiac_arrest)/n(),
      stroke=sum(na.rm=T,stroke)/n(),
      mean_weighted_charlson=mean(wscore,na.rm=T),
      number_of_hospitalizations=sum(na.rm=T,number_of_hospitalizations)/n(),
      hospitalized_patients=sum(na.rm=T,was_hospitalized)/n()
      ,catheterization=sum(na.rm=T,catheterization_count>0)/n()
      ,catheterization_count=sum(na.rm=T,catheterization_count)/n()
      ,catheterization_cost=sum(na.rm=T,catheterization_cost)/n()
      ,ecg=sum(na.rm=T,ecg_count>0)/n()
      ,ecg_count=sum(na.rm=T,ecg_count)/n()
      ,ecg_cost=sum(na.rm=T,ecg_cost)/n()
      ,cardiac_ct=sum(na.rm=T,cardiac_ct_count>0)/n()
      ,cardiac_ct_count=sum(na.rm=T,cardiac_ct_count)/n()
      ,cardiac_ct_cost=sum(na.rm=T,cardiac_ct_cost)/n()
      ,cardiac_mri=sum(na.rm=T,cardiac_mri_count>0)/n()
      ,cardiac_mri_count=sum(na.rm=T,cardiac_mri_count)/n()
      ,cardiac_mri_cost=sum(na.rm=T,cardiac_mri_cost)/n()
      ,stress_test=sum(na.rm=T,stress_test_count>0)/n()
      ,stress_test_count=sum(na.rm=T,stress_test_count)/n()
      ,stress_test_cost=sum(na.rm=T,stress_test_cost)/n()
      ,echocardiography=sum(na.rm=T,echocardiography_count>0)/n()
      ,echocardiography_count=sum(na.rm=T,echocardiography_count)/n()
      ,echocardiography_cost=sum(na.rm=T,echocardiography_cost)/n()
      ,angioplasty=sum(na.rm=T,angioplasty_count>0)/n()
      ,angioplasty_count=sum(na.rm=T,angioplasty_count)/n()
      ,angioplasty_cost=sum(na.rm=T,angioplasty_cost)/n()
      ,CABG=sum(na.rm=T,CABG_count>0)/n()
      ,CABG_count=sum(na.rm=T,CABG_count)/n()
      ,CABG_cost=sum(na.rm=T,CABG_cost)/n(),
      most_common_physician_is_integrated=sum(na.rm=T,most_common_physician_is_integrated)/n(),
      most_common_primary_care_physician_is_integrated=sum(na.rm=T,most_common_primary_care_physician_is_integrated)/n(),
      most_common_cardiologist_is_integrated=sum(na.rm=T,most_common_cardiologist_is_integrated)/n(),
      most_common_interventional_cardiologist_is_integrated=sum(na.rm=T,most_common_interventional_cardiologist_is_integrated)/n(),
    )%>%
    as.data.table()
  setnames(result, "eval(parse(text = grouping_var))", grouping_var)
  return(as.data.table(result))
}

In [ ]:
#comparisons for stable angina patients
stable_angina_physician_comparisons=intensity_comparator(yearly_calculations_stable_angina[year_first_diagnosis!=2013],"most_common_physician_is_integrated")
write.csv(stable_angina_physician_comparisons,"stable_angina_physician_comparisons.csv")

stable_angina_primary_care_comparisons=intensity_comparator(yearly_calculations_stable_angina[year_first_diagnosis!=2013],"most_common_primary_care_physician_is_integrated")
write.csv(stable_angina_primary_care_comparisons,"stable_angina_primary_care_comparisons.csv")

stable_angina_cardiologist_comparisons=intensity_comparator(yearly_calculations_stable_angina[year_first_diagnosis!=2013],"most_common_cardiologist_is_integrated")
write.csv(stable_angina_cardiologist_comparisons,"stable_angina_cardiologist_comparisons.csv")

stable_angina_interventional_cardiologist_comparisons=intensity_comparator(yearly_calculations_stable_angina[year_first_diagnosis!=2013],"most_common_interventional_cardiologist_is_integrated")
write.csv(stable_angina_interventional_cardiologist_comparisons,"stable_angina_interventional_cardiologist_comparisons.csv")


#comparisons for unstable angina patients
unstable_angina_physician_comparisons=intensity_comparator(yearly_calculations_unstable_angina[year_first_diagnosis!=2013],"most_common_physician_is_integrated")
write.csv(unstable_angina_physician_comparisons,"unstable_angina_physician_comparisons.csv")

unstable_angina_primary_care_comparisons=intensity_comparator(yearly_calculations_unstable_angina[year_first_diagnosis!=2013],"most_common_primary_care_physician_is_integrated")
write.csv(unstable_angina_primary_care_comparisons,"unstable_angina_primary_care_comparisons.csv")

unstable_angina_cardiologist_comparisons=intensity_comparator(yearly_calculations_unstable_angina[year_first_diagnosis!=2013],"most_common_cardiologist_is_integrated")
write.csv(unstable_angina_cardiologist_comparisons,"unstable_angina_cardiologist_comparisons.csv")

unstable_angina_interventional_cardiologist_comparisons=intensity_comparator(yearly_calculations_unstable_angina[year_first_diagnosis!=2013],"most_common_interventional_cardiologist_is_integrated")
write.csv(unstable_angina_interventional_cardiologist_comparisons,"unstable_angina_interventional_cardiologist_comparisons.csv")